## Análisis del comportamiento del Covid-19 en la ciudad de Maracaibo - Junio 2020

In [13]:
import pandas as pd
import iplotter
import datetime
import matplotlib.pyplot as plt
import numpy as np

In [14]:
columns=['FECHA', 'CANTIDAD PRUEBAS', 'POSITIVOS PDR', 'INGRESOS', 'EGRESOS POR SINTOMAS','CASOS POSITIVOS', 'CASOS NEGATIVOS', 'DE ALTA', 'VIGILANCIA EPIDEMIOLOGICA']
df = pd.read_excel('Covid-Mcbo.xlsx', names = columns)

df.head()

,FECHA,CANTIDAD PRUEBAS,POSITIVOS PDR,INGRESOS,EGRESOS POR SINTOMAS,CASOS POSITIVOS,CASOS NEGATIVOS,DE ALTA,VIGILANCIA EPIDEMIOLOGICA
0,2020-05-22,424.0,10,0.0,0,0,0,277,404
1,2020-05-23,51.0,3,0.0,0,0,0,0,404
2,2020-05-24,35.0,2,0.0,0,0,0,0,404
3,2020-05-25,286.0,3,0.0,0,0,0,0,404
4,2020-05-26,85.0,3,0.0,0,0,0,0,404


In [15]:
# Total de casos confirmados al 11 de junio

In [16]:
print("\nTotal de casos confirmados al 11 de junio: ", df['CASOS POSITIVOS'].sum(), "\n")


Total de casos confirmados al 11 de junio:  380 



In [17]:
# Chequeamos tipo de datos 

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
FECHA                        21 non-null datetime64[ns]
CANTIDAD PRUEBAS             19 non-null float64
POSITIVOS PDR                21 non-null int64
INGRESOS                     20 non-null float64
EGRESOS POR SINTOMAS         21 non-null int64
CASOS POSITIVOS              21 non-null int64
CASOS NEGATIVOS              21 non-null int64
DE ALTA                      21 non-null int64
VIGILANCIA EPIDEMIOLOGICA    21 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 1.6 KB


In [19]:
# Agrupamos por fecha los Casos PDR Positivos, PCR Positivos y Casos PCR Negativos

df_dates = df.groupby(['FECHA'])[['POSITIVOS PDR', 'CASOS POSITIVOS', 'CASOS NEGATIVOS']].sum().reset_index()

# Put the result in a dataframe

df_dates = pd.DataFrame(df_dates)

data = {    
    'labels': df_dates['FECHA'].astype(str).tolist(),
    'datasets': [{ 
        'data': df_dates['POSITIVOS PDR'].values.tolist(),
        'label': "Casos PDR Positivos",
        'borderColor': "#FFCE00"
      }, { 
        'data': df_dates['CASOS POSITIVOS'].values.tolist(),
        'label': "Casos PCR Positivos",
        'borderColor': "#E63029"
      },
      { 
        'data': df_dates['CASOS NEGATIVOS'].values.tolist(),
        'label': "Casos Negativos (De Alta)",
        'borderColor': "#007849"
      }
    ]
  }

chart_js = iplotter.ChartJSPlotter()
chart_js.plot(data, chart_type="line")

C:\Users\WAKU\Anaconda3\lib\site-packages\IPython\core\display.py:694: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [20]:
# formatted_df = df["Date"].dt.strftime("%m/%d/%y")
# df.index = pd.to_datetime(df.index, dayfirst=True)
# df_dates = df_dates.dt.strftime("%m/%d/%y")

df['FECHA'] = df['FECHA'].apply(pd.to_datetime)

# Check whether the Date data type was successfully changed to datetime64

print("\n")
df.info()

# Access date element from Date column

df['FECHA'] = df['FECHA'].dt.date

df.head()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 9 columns):
FECHA                        21 non-null datetime64[ns]
CANTIDAD PRUEBAS             19 non-null float64
POSITIVOS PDR                21 non-null int64
INGRESOS                     20 non-null float64
EGRESOS POR SINTOMAS         21 non-null int64
CASOS POSITIVOS              21 non-null int64
CASOS NEGATIVOS              21 non-null int64
DE ALTA                      21 non-null int64
VIGILANCIA EPIDEMIOLOGICA    21 non-null int64
dtypes: datetime64[ns](1), float64(2), int64(6)
memory usage: 1.6 KB


,FECHA,CANTIDAD PRUEBAS,POSITIVOS PDR,INGRESOS,EGRESOS POR SINTOMAS,CASOS POSITIVOS,CASOS NEGATIVOS,DE ALTA,VIGILANCIA EPIDEMIOLOGICA
0,2020-05-22,424.0,10,0.0,0,0,0,277,404
1,2020-05-23,51.0,3,0.0,0,0,0,0,404
2,2020-05-24,35.0,2,0.0,0,0,0,0,404
3,2020-05-25,286.0,3,0.0,0,0,0,0,404
4,2020-05-26,85.0,3,0.0,0,0,0,0,404


## Construimos gráfico mostrando los Casos PDR Positivos y la Proyección de estos en el tiempo, utilizando el algoritnmo de los promedios móviles (moving average).

In [25]:
# Add a column to the dataframe including the moving average with of 2

# df_dates = pd.to_datetime(df_dates, format='%Y%m%d', errors='ignore')
# df_dates = df_dates.dt.strftime("%m/%d/%y")
df_dates['MA'] = df_dates.iloc[:,1].rolling(window=2).mean()

# Calculate the moving average for 4 days ahead using the last moving average and the last data point

predictions = [int((df_dates['MA'].values.tolist()[-1])), int((df_dates['POSITIVOS PDR'].values.tolist()[-1] + df_dates['MA'].values.tolist()[-1]) / 2)]

for i in range(0, 4):
    for j in zip(predictions, predictions[1:]):
        ma = int((j[0] + j[1]) / 2)
    predictions.append(ma)

# Get the next 6 dates and assign them with their predictions

df_predicted = pd.DataFrame(df_dates.FECHA + datetime.timedelta(days=6)).iloc[-6:]
df_predicted['Predicted'] = predictions

# Merge all the dates into a list for graphing

all_dates = df_dates['FECHA'].astype(str).tolist()
all_dates.extend(df_predicted['FECHA'].astype(str).tolist())

# Preparing predicted data for graphing

predicted_data = ["NaN"] * 20
predicted_data.append(df_dates['POSITIVOS PDR'].values.tolist()[-1])
predicted_data.extend(df_predicted['Predicted'].values.tolist())

# Graph the data 

data = {    
    'labels': all_dates,
    'datasets': [{ 
        'data': df_dates['POSITIVOS PDR'].values.tolist(),
        'label': "Casos PDR Positivos",
        'borderColor': "#FFCE00"
      },
        { 
        'data': predicted_data,
        'label': "Predicción",
        'borderColor': "#E63029"
      },
        { 
        'data': df_dates['MA'].values.tolist(),
        'label': "Promedio móvil (Moving Average)",
        'borderColor': "#1307ED"
      }
    ]
  }

chart_js = iplotter.ChartJSPlotter()
chart_js.plot(data, chart_type="line")